In [1]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import urllib2
from multiprocessing import Pool as ThreadPool
import math
import cPickle
import os 

In [2]:
priceArr = []
patCollect = []
endingInd = []
patLen = 10
futureE = 10

def percentChange(startPoint,currentPoint):
    try:
        x = ((float(currentPoint)-startPoint)/abs(startPoint))
        return x
    except:
        print(currentPoint, startPoint)
        return 0.00000000001

def loadPats():
    global priceArr
    
    priceArr = []
    with open(r"one.pickle", "rb") as input_file:
        e = cPickle.load(input_file)
        priceArr = e


def collectPats():
    global patCollect
    global endingInd
    patCollect = []
    endingInd = []
    for each in priceArr:
        tempCollect = []
        tempEnd = []
        sIndex = 0
        length = (len(each) - patLen - futureE)

        while sIndex < length:
            inc = 1
            tempPat = []
            while inc <= patLen:
                temp = percentChange(each[sIndex + inc -1], each[sIndex + inc])
                inc += 1
                tempPat.append(temp)
            tempCollect.append(tempPat)
            tempEnd.append(sIndex+patLen)
            sIndex += (patLen)
        patCollect.append(tempCollect)
        endingInd.append(tempEnd)

loadPats()
collectPats()

In [23]:
def yahooLoad(val):
    tempArr = []
    string = ""
    string = 'http://ichart.finance.yahoo.com/table.csv?s={0}'.format(val)
    count = 0
    for line in urllib2.urlopen(string):
        if count == 0:
            count = 1
            continue
        close = line.split(',')[6]
        close = float(str(close))
        tempArr.append(close)
    tempArr = tempArr[::-1]
    #quoteCollect[val] = tempArr
    return {val: tempArr}


def currentPat(curArr):
    global patLen
    curPat = []
    sliceLen = patLen
    curr = curArr[-(sliceLen+1):]
    while curr[-2] == curr[-1]:
        sliceLen += 1
        curr = curArr[-(sliceLen+1):]
    i = 0
    while i < patLen:
        temp = percentChange(curr[i], curr[i + 1])
        curPat.append(temp)
        i += 1
    return curPat


def sortPats(array):
    finalArr = []
    def getKey(tup):
        return tup[1]
    finalArr = sorted(array, key=getKey)
    return finalArr

In [24]:
def matchPats(patCollect,endingInd,curPat):
    matchedPat = []
    matchedEndInd = []
    bestMatches = []
    for colInd,col in enumerate(patCollect):
        for rowInd,row in enumerate(col):
            mseCollect = []
            for ind,item in enumerate(row):
                mseTemp = abs(curPat[ind] - item)
                mseCollect.append(mseTemp)
            #mseAvg = np.average(weighting)
            mseAvg = np.average(mseCollect)
            bestMatches.append(([row,colInd,endingInd[colInd][rowInd]],mseAvg))
    test = sortPats(bestMatches)
    for item in test[:10]:
        matchedPat.append(item[0][0])
        matchedEndInd.append((item[0][1],item[0][2]))
    return matchedPat,matchedEndInd

In [25]:
def plotting(matchedPat,matchedEndInd):
    global futureE
    futureAverages = []
    stDev = []
    futureLine = []
    futurePercent = []
    if len(matchedPat) > 1:
        for i in matchedPat:
            tempPercent = []
            refInd = matchedPat.index(i)
            endInd = matchedEndInd[refInd]
            col = endInd[0]
            outcomesZ = priceArr[col][endInd[1]:(endInd[1]+patLen)]
            futureLine.append(outcomesZ)
            for x in outcomesZ:
                change = percentChange(priceArr[col][endInd[1]],x)
                tempPercent.append(change)
            futurePercent.append(tempPercent)

        futureAverages = []

        for arrItemNum in range(len(futurePercent[0])):
            tempOutcomes = []
            for arr in futurePercent:
                tempOutcomes.append(arr[arrItemNum])
            futureAverages.append(np.mean(tempOutcomes))
            stDev.append(np.std(tempOutcomes))

        return futureAverages,stDev

In [26]:
def sortBest(array):
    finalArr = []
    def getKey(val):
        return val['future'][-1]
    finalArr = sorted(array, key=getKey, reverse=True)
    return finalArr

bestCollect = []

def bestGo(val,curArr):
    global bestCollect
    try:
        #curArr = yahooLoad(val)
        curPat = currentPat(curArr)
        matchedPat,matchedEndInd = matchPats(patCollect,endingInd,curPat)
        futureAverages,stDev = plotting(matchedPat,matchedEndInd)
        totalDict = {'symbol': val,'matches': matchedPat,'current': curPat,'future': futureAverages, 'stDev': stDev, 'sharpe': futureAverages[-1]/stDev[-1]}
        bestCollect.append(totalDict)
    except:
        bestCollect.append({'error':'error'})
        pass

In [27]:
yahooLoad('AAPL')

<addinfourl at 4423225000 whose fp = <socket._fileobject object at 0x107a38450>>


{'AAPL': [0.431358,
  0.408852,
  0.378845,
  0.388222,
  0.399475,
  0.423856,
  0.444486,
  0.463241,
  0.487622,
  0.532633,
  0.540135,
  0.527007,
  0.512003,
  0.51763,
  0.506377,
  0.483871,
  0.463241,
  0.453864,
  0.478245,
  0.474494,
  0.457615,
  0.45949,
  0.468867,
  0.465116,
  0.493248,
  0.478245,
  0.487622,
  0.493248,
  0.491373,
  0.483871,
  0.48012,
  0.465116,
  0.448237,
  0.423856,
  0.399475,
  0.414479,
  0.429483,
  0.429483,
  0.431358,
  0.408852,
  0.408852,
  0.395724,
  0.391973,
  0.382596,
  0.391973,
  0.408852,
  0.384471,
  0.363841,
  0.369467,
  0.356339,
  0.378845,
  0.384471,
  0.3976,
  0.399475,
  0.393849,
  0.390098,
  0.388222,
  0.384471,
  0.354464,
  0.337584,
  0.324456,
  0.337584,
  0.333834,
  0.346962,
  0.363841,
  0.386347,
  0.382596,
  0.386347,
  0.40135,
  0.399475,
  0.391973,
  0.384471,
  0.371343,
  0.371343,
  0.367592,
  0.363841,
  0.395724,
  0.3976,
  0.390098,
  0.386347,
  0.405101,
  0.412603,
  0.41823,
  0.4

In [7]:
arr = ['MMM','ABT','ABBV','ACN','ATVI','AYI','ADBE','AAP','AES','AET','AMG','AFL','A','GAS','APD','AKAM','ALK','AA','ALXN','ALLE','AGN','ADS','ALL','GOOGL','GOOG','MO','AMZN','AEE','AAL','AEP','AXP','AIG','AMT','AWK','AMP','ABC','AME','AMGN','APH','APC','ADI','ANTM','AON','APA','AIV','AAPL','AMAT','ADM','AJG','AIZ','T','ADSK','ADP','AN','AZO','AVGO','AVB','AVY','BHI','BLL','BAC','BCR','BAX','BBT','BDX','BBBY','BRK-B','BBY','BIIB','BLK','HRB','BA','BWA','BXP','BSX','BMY','BF-B']

In [8]:
pool = ThreadPool(4)
quoteCollect = pool.map(yahooLoad, arr)
pool.close()
pool.join()




In [10]:
bestCollect = []
for i in quoteCollect:
    for key,value in i.iteritems():
        bestGo(key,value)

finalBest = []
for val in bestCollect:
    if val['sharpe'] > 0.75:
        finalBest.append(val)

bestArr = sortBest(finalBest)

#os.remove("best.pickle")
with open(r"best.pickle", "wb") as output_file:
    cPickle.dump(bestArr, output_file)

In [11]:
t = []
with open(r"best.pickle", "rb") as input_file:
        t = cPickle.load(input_file)
print(t)

[{'matches': [[-0.016976869084129907, -0.024180633600434875, -0.005309268978070864, -0.005337439453052913, -0.0035789100679754823, -0.017952199928263298, 0.00182758765001768, -0.003649715676739673, 0.00549402095770354, 0.01639390022299873], [-0.016760736071312645, -0.0200034711667343, 0.00443733129102164, -0.004064366543725515, 0.00035488046613383587, -0.004434183303642848, -0.0021379692902386035, -0.001606806527708366, 0.018240347861595733, 0.01246934101824258], [-0.009956690119990735, -0.001311791028041323, -0.002626947475467547, 0.0035118489742860795, 0.00021870060558951292, -0.005466883882903786, 0.010554079516460224, 0.001958241050945184, 0.004994547919132336, 0.012748503575860269], [-0.006422543558848566, -0.024240464633469145, -0.002318648482323677, -0.0029880477095601643, -0.010989010623076513, 0.001346767631421902, 0.008742434431741814, -0.005333333333333338, 0.0130697050938338, 0.009923949718822302], [-0.013783873754018348, -0.014427406659234981, 0.004345814920249352, -0.0025

In [ ]:
print(quoteCollect[0])